# Canopy height

In [1]:
# Vector data
import geopandas as gpd

# Tabular data 
import pandas as pd
import numpy as np

# Raster array
import xarray as xr

# Plotting
import matplotlib.pyplot as plt

# System
from pathlib import Path

# Import functions from geo_tools module
from geo_tools import plot_iso_forest

# Reloads module
%load_ext autoreload
%autoreload 2

# Testing forest height

In [ ]:
deu_avg_height = plot_iso_forest.average_values(iso_code="DEU", xarray_id="Canopy_height", forest_layer_year=2020)

In [ ]:
deu_avg_height